In [7]:
import rasterio
from pathlib import Path

def check_pixel_size_tif_precise(tif_path):
    with rasterio.open(tif_path) as src:
        transform = src.transform
        pixel_width = transform.a
        pixel_height = -transform.e  # make positive
        crs = src.crs
        
        print(f"File: {tif_path}")
        print(f"CRS: {crs}")
        print(f"Pixel size (width): {pixel_width:.8f} {'m' if crs.is_projected else 'degrees'}")
        print(f"Pixel size (height): {pixel_height:.8f} {'m' if crs.is_projected else 'degrees'}")

# Example usage
lulc_file = Path(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped.tif")
check_pixel_size_tif_precise(lulc_file)


File: C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped.tif
CRS: EPSG:4326
Pixel size (width): 0.00416667 degrees
Pixel size (height): 0.00416667 degrees


In [6]:
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np 

In [15]:
# Input files
lulc_file = r"C:\Users\Ankit\Datasets_Forest_fire\lulc_maps_tif\LULC_2016_clipped.tif"
viirs_file = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack.tif"
out_file = r"C:\Users\Ankit\Datasets_Forest_fire\LULC_2016_resampled_to_VIIRS.tif"

In [16]:
# Open reference (VIIRS grid)
with rasterio.open(viirs_file) as ref:
    ref_transform = ref.transform
    ref_crs = ref.crs if ref.crs is not None else "EPSG:4326"   # 🔑 force CRS
    ref_height = ref.height
    ref_width = ref.width

In [17]:
# Open LULC raster
with rasterio.open(lulc_file) as src:
    profile = src.profile.copy()
    profile.update(
        driver="GTiff",
        height=ref_height,
        width=ref_width,
        transform=ref_transform,
        crs=ref_crs,
        count=src.count,
        dtype=src.dtypes[0],
        compress="lzw"
    )

    # Allocate destination
    resampled = np.empty((src.count, ref_height, ref_width), dtype=src.dtypes[0])

    # Reproject using nearest neighbour (for categorical LULC)
    reproject(
        source=rasterio.band(src, 1),
        destination=resampled[0],
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=ref_transform,
        dst_crs=ref_crs,
        resampling=Resampling.nearest
    )

    # Save
    with rasterio.open(out_file, "w", **profile) as dst:
        dst.write(resampled)

print(f"✅ LULC resampled to VIIRS grid → {out_file}")

✅ LULC resampled to VIIRS grid → C:\Users\Ankit\Datasets_Forest_fire\LULC_2016_resampled_to_VIIRS.tif
